<a href="https://colab.research.google.com/github/shadab007-byte/Imdb_senrimental_analysis-using-LSTM/blob/main/lstm_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install nltk scikit-learn numpy pandas -q

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Dense , Embedding , LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [ ]:
from zipfile import ZipFile
import os
import json

kaggle_dic = json.load(open('kaggle.json'))
os.environ['KAGGLE_USERNAME'] = kaggle_dic['username']
os.environ['KAGGLE_KEY'] = kaggle_dic['key']

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.36GB/s]


In [ ]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [ ]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip' , 'r') as zip_ref:
  zip_ref.extractall('data')


In [ ]:
data = pd.read_csv('data/IMDB Dataset.csv')

In [ ]:
data.sample(5)

,review,sentiment
30961,"I don't normally write reviews, but for this f...",negative
49827,Originally I rented this film for my daughter ...,positive
42191,One of the perks of my job is that when things...,positive
23187,If you thought this is the french The Mummy an...,negative
35215,I'm gonna tip the scales here a bit and say I ...,positive


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
data['sentiment'] = data['sentiment'].apply(lambda x : 1 if x == 'positive' else 0)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
train_df , test_df = train_test_split(data , test_size = 0.3 , random_state=42)

In [ ]:
train_df

,review,sentiment
38094,"As much as I love trains, I couldn't stomach t...",0
40624,"This was a very good PPV, but like Wrestlemani...",1
49425,Not finding the right words is everybody's pro...,0
35734,I'm really suprised this movie didn't get a hi...,1
41708,I'll start by confessing that I tend to really...,0
...,...,...
11284,`Shadow Magic' recaptures the joy and amazemen...,1
44732,I found this movie to be quite enjoyable and f...,1
38158,Avoid this one! It is a terrible movie. So wha...,0
860,This production was quite a surprise for me. I...,1


Data preprocessing

In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_df['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_df['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_df['review']), maxlen=200)

In [ ]:
X_test

array([[   0,    0,    0, ...,  986,  722,  155],
       [ 133,    6,  115, ...,    7, 8994, 8537],
       [   0,    0,    0, ...,   51, 1092,   98],
       ...,
       [  26,  151,   45, ...,   66,    3,  598],
       [   0,    0,    0, ..., 2047,   10,   67],
       [   0,    0,    0, ...,   64,   11,   19]], dtype=int32)

In [ ]:
X_train

array([[   0,    0,    0, ...,   62,  214,  159],
       [   0,    0,    0, ...,  450, 1227,  501],
       [   0,    0,    0, ...,  646, 1075,    4],
       ...,
       [   0,    0,    0, ..., 1624,    2,  606],
       [   0,    0,    0, ...,  103,  125, 7314],
       [   0,    0,    0, ...,   70,   71, 2071]], dtype=int32)

In [ ]:
y_train = train_df['sentiment']
y_test = test_df['sentiment']

In [ ]:
LSTM

keras.src.layers.rnn.lstm.LSTM

In [ ]:
# build the model

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(X_train , y_train, batch_size = 32, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 771s 697ms/step - accuracy: 0.7371 - loss: 0.5153 - val_accuracy: 0.8635 - val_loss: 0.3353
Epoch 2/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 766s 700ms/step - accuracy: 0.8558 - loss: 0.3423 - val_accuracy: 0.8702 - val_loss: 0.3230
Epoch 3/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 787s 686ms/step - accuracy: 0.8952 - loss: 0.2598 - val_accuracy: 0.8661 - val_loss: 0.3298
Epoch 4/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 800s 685ms/step - accuracy: 0.9084 - loss: 0.2326 - val_accuracy: 0.8893 - val_loss: 0.3011
Epoch 5/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 801s 684ms/step - accuracy: 0.9419 - loss: 0.1553 - val_accuracy: 0.8815 - val_loss: 0.3258


In [ ]:
Evaluation = model.evaluate(X_test , y_test)
Evaluation

469/469 ━━━━━━━━━━━━━━━━━━━━ 54s 115ms/step - accuracy: 0.8805 - loss: 0.3216


[0.3258022964000702, 0.8814666867256165]

In [ ]:
from collections.abc import Sequence
def prediction_of_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  if prediction[0][0] > 0.5:
    return 'positive'
  else:
    return 'negative'

In [ ]:
review = "I like the Movie "
prediction_of_sentiment(review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step


'positive'

In [ ]:
import pickle

model.save('sentiment_model.h5')
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [ ]:
!pip install flask pyngrok
import pickle
from tensorflow.keras.models import load_model


In [ ]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import numpy as np
from keras.preprocessing.sequence import pad_sequences

# -------------------------------
# Load your model and tokenizer
# -------------------------------
model = load_model('sentiment_model.h5')  # Already loaded in your case
with open('tokenizer.pkl', 'rb') as f:
  tokenizer = pickle.load(f)             # Make sure tokenizer is defined

# Flask app setup
app = Flask(__name__)

# Start ngrok
ngrok.kill()  # Clean up existing tunnels
ngrok.set_auth_token("token")  # Replace with your token
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

# -------------------------------
# Prediction Function
# -------------------------------
def prediction_of_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    if prediction[0][0] > 0.5:
        return 'Positive'
    else:
        return 'Negative'

# -------------------------------
# HTML Template
# -------------------------------
html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>📝 Sentiment Predictor</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body {
            background: linear-gradient(to right, #2c3e50, #3498db);
            color: white;
            font-family: 'Segoe UI', sans-serif;
            padding-top: 70px;
        }
        .container {
            max-width: 600px;
            margin: auto;
            background: rgba(255, 255, 255, 0.1);
            border-radius: 15px;
            padding: 30px;
            box-shadow: 0 8px 16px rgba(0,0,0,0.3);
        }
        h1 {
            font-weight: bold;
            margin-bottom: 30px;
        }
        .result {
            margin-top: 20px;
            font-size: 1.4em;
            font-weight: bold;
        }
    </style>
</head>
<body>
    <div class="container text-center">
        <h1>📝 Sentiment Predictor</h1>
        <form method="POST">
            <textarea name="review" rows="5" class="form-control" placeholder="Enter your review..." required></textarea>
            <button type="submit" class="btn btn-warning mt-3">Predict</button>
        </form>
        {% if prediction %}
        <div class="result">
            Prediction: <span class="text-white">{{ prediction }}</span>
        </div>
        {% endif %}
    </div>
</body>
</html>
'''

# -------------------------------
# Flask Routes
# -------------------------------
@app.route('/', methods=['GET', 'POST'])
def index():
    prediction = None
    if request.method == 'POST':
        review = request.form['review']
        prediction = prediction_of_sentiment(review)
    return render_template_string(html_template, prediction=prediction)

# -------------------------------
# Run the Flask app
# -------------------------------
app.run()


Public URL: NgrokTunnel: "https://faed-34-83-90-137.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 09:53:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 09:53:58] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step


INFO:werkzeug:127.0.0.1 - - [16/Jun/2025 09:54:17] "POST / HTTP/1.1" 200 -
